<a href="https://colab.research.google.com/github/Matidonoso/CRYPTOgo/blob/master/CRYPTOgo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

import json
import requests
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error
%matplotlib inline
######################
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
from tensorflow.python.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential

https://www.cryptocompare.com/coins/guides/how-to-use-our-api/

In [0]:
from google.colab import drive
drive.mount("mnt")

ValueError: ignored

In [0]:
%cd "mnt/My Drive/Startup School 2020/RAIs/RAIs 2020-1/Proyectos/Fintech"

In [0]:
!pip install import-ipynb
import import_ipynb

In [0]:
import fetch_market_data

TIME_DELTA = 1
hours = 80 * 24 #DAYS PER HOURS

In [0]:
def plot_close_price(dataframe):
  
  ax = dataframe.plot(y='close')
  ax.set_xlabel("Time"),
  ax.set_ylabel("Close Price (USD)")


In [0]:
#BITCOIN DATAFRAME & PLOT
BTC_DF = fetch_market_data.get_historical_hourly_price('BTC', 'USD', hours, TIME_DELTA)
plot_close_price(BTC_DF)

In [0]:
#ETHEREUM DATAFRAME & PLOT
ETH_DF = fetch_market_data.get_historical_hourly_price('ETH', 'USD', hours, TIME_DELTA)

plot_close_price(ETH_DF)

In [0]:
#STELLAR DATAFRAME & PLOT
XLM_DF = fetch_market_data.get_historical_hourly_price('XLM', 'USD', hours, TIME_DELTA)

plot_close_price(XLM_DF)

# **Normalization**

In [0]:
# We’re going to use the MinMaxScaler from scikit learn:
scaler = MinMaxScaler()

def normalization(dataframe):

  # The scaler expects the data to be shaped as (x, y), so we add a dummy dimension using reshape before applying it.
  close_price = dataframe.close.values.reshape(-1,1)

  scaled_close = scaler.fit_transform(close_price)

  # Let’s also remove NaNs since our model won’t be able to handle them well
  scaled_close = scaled_close[~np.isnan(scaled_close)]
  
  scaled_close = scaled_close.reshape(-1, 1)

  return scaled_close



In [0]:
#BITCOIN NORMALIZATION
BTC_DF = normalization(BTC_DF)

In [0]:
#ETHEREUM NORMALIZATION
ETH_DF = normalization(ETH_DF)

In [0]:
#STELLAR NORMALIZATION
XLM_DF = normalization(XLM_DF)

# **Preprocessing**


In [0]:
SEQ_LEN = 100

def to_sequences(data, seq_len):
    d = []

    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])

    return np.array(d)

def preprocess(data_raw, seq_len, train_split):

    data = to_sequences(data_raw, seq_len)

    num_train = int(train_split * data.shape[0])

    X_train = data[:num_train, :-1, :]
    y_train = data[:num_train, -1, :]

    X_test = data[num_train:, :-1, :]
    y_test = data[num_train:, -1, :]

    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test =\
preprocess(XLM_DF, SEQ_LEN, train_split = 0.95)

# **Model**

In [0]:
DROPOUT = 0.2
WINDOW_SIZE = SEQ_LEN - 1

def build_model(X_train, DROPOUT, WINDOW_SIZE):

  model = keras.Sequential()

  model.add(Bidirectional(CuDNNLSTM(WINDOW_SIZE, return_sequences=True),
                          input_shape=(WINDOW_SIZE, X_train.shape[-1])))
  model.add(Dropout(rate=DROPOUT))

  model.add(Bidirectional(CuDNNLSTM((WINDOW_SIZE * 2), return_sequences=True)))
  model.add(Dropout(rate=DROPOUT))

  model.add(Bidirectional(CuDNNLSTM(WINDOW_SIZE, return_sequences=False)))

  model.add(Dense(units=1))

  model.add(Activation('linear'))

  return model

# **Training**

In [0]:
BATCH_SIZE = 64

def compile_and_train(X_train, y_train, BATCH_SIZE):
  model.compile(
      loss='mean_squared_error',
      optimizer='adam'
  )

  history = model.fit(
      X_train,
      y_train,
      epochs=50,
      batch_size=BATCH_SIZE,
      shuffle=False,
      validation_split=0.1
  )

  return history

In [0]:
model.evaluate(X_test, y_test)

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# **Prediction**

In [0]:

def prediction(X_test , y_test , crypto):
  y_hat = model.predict(X_test)

  y_test_inverse = scaler.inverse_transform(y_test)
  y_hat_inverse = scaler.inverse_transform(y_hat)
  
  plt.plot(y_test_inverse, label="Actual Price", color='green')
  plt.plot(y_hat_inverse, label="Predicted Price", color='red')
  
  plt.title('{} price prediction'.format(crypto))
  plt.xlabel('Hours')
  plt.ylabel('Price')
  plt.legend(loc='best')
  
  return plt.show();

# UI express


In [0]:
print("What Cryptocurrency do you want to predict? (BTC, ETH, XLM)")
request = input()
print('You requested: {}'.format(request))

user_request = fetch_market_data.get_historical_hourly_price(request, 'USD', hours, TIME_DELTA)
plot_close_price(user_request)
u_r = normalization(user_request)
preprocess(u_r, SEQ_LEN, train_split = 0.95)
build_model(X_train,DROPOUT,WINDOW_SIZE)
compile_and_train(X_train, y_train)
prediction(X_test,y_test,request)


What Cryptocurrency do you want to predict? (BTC, ETH, XLM)
